In [6]:
import os.path
import zipfile
fname = 'discogs_mixed.csv'
if not os.path.isfile(fname):
    zip_ref = zipfile.ZipFile(fname+".zip", 'r')
    zip_ref.extractall('.')
    zip_ref.close()
with open(fname) as f:
    print(f.read().split('\n',2)[:2])

[',artist,cat_no,country,genre,label_root,labels,length,position,release,release_artist,release_date,release_descriptions,styles,track,year,bool_R_mixed,bool_R_electronic,bool_R_Abstract,bool_R_Acid,bool_R_Deep,bool_R_Disco,bool_R_Drum_n_Bass,bool_R_Dub,bool_R_Electro,bool_R_Experimental,bool_R_Funk,bool_R_Garage,bool_R_Hard,bool_R_Hip_Hop,bool_R_House,bool_R_IDM,bool_R_Industrial,bool_R_Leftfield,bool_R_Minimal,bool_R_Progressive,bool_R_Psy-Trance,bool_R_Tech_House,bool_R_Techno,bool_R_Trance', '0,Vector,EKCY-10001,Japan,[\'Electronic\'],Planet Rhythm Records,"[\'EPS\', \'Planet Rhythm Records\']",,1,Planet Rhythm Label Collection Vol.001,Various,18 Jun 1997,[\'Mixed\'],[\'Minimal Techno\'],Turbulence,1997.0,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False']


In [11]:
boooools = "bool_R_mixed,bool_R_electronic,bool_R_Abstract,bool_R_Acid,bool_R_Deep,bool_R_Disco,bool_R_Drum_n_Bass,bool_R_Dub,bool_R_Electro,bool_R_Experimental,bool_R_Funk,bool_R_Garage,bool_R_Hard,bool_R_Hip_Hop,bool_R_House,bool_R_IDM,bool_R_Industrial,bool_R_Leftfield,bool_R_Minimal,bool_R_Progressive,bool_R_Psy-Trance,bool_R_Tech_House,bool_R_Techno,bool_R_Trance"\
            .split(',')

feature_amount = len(boooools)
print(feature_amount)

24


In [7]:
import pandas
tracklist_df = pandas.read_csv(fname)

In [8]:
def get_tracklists(df):
    tracklist = []
    album = ""
    for i, track in df.iterrows():
        if album == "":
            album = track['release']
        if track['release'] != album:
            yield album, tracklist
            album = track['release']
            tracklist = []
        tracklist += [track]
    yield album, tracklist


def to_vector(track):
    return [int(bool(track[b])) for b in boooools]

def to_matrix(tracklist):
    return [to_vector(track) for track in tracklist]

def seq_matrices(df):
    for album, tracklist in get_tracklists(df):
        yield to_matrix(tracklist)

In [9]:
def group_matrices_by_length(df):
    groups = {}
    for m in seq_matrices(df):
        if len(m) in groups:
            groups[len(m)] += [m]
        else:
            groups[len(m)] = [m]
    return groups

In [12]:
groups = group_matrices_by_length(tracklist_df)

In [13]:

from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense

model = Sequential()
# batch, time_steps, features
# input_shape = (time_steps, features)
# time_steps = variable = -1 or None
model.add(LSTM(200, input_shape=(None, feature_amount), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(200, return_sequences=True))
model.add(Dense(feature_amount))
print(model.summary())

model.compile(optimizer="adam", loss="MSE")


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 200)         180000    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 200)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 200)         320800    
_________________________________________________________________
dense_1 (Dense)              (None, None, 24)          4824      
Total params: 505,624
Trainable params: 505,624
Non-trainable params: 0
_________________________________________________________________
None


In [14]:
import numpy as np

print(len(groups.items()))
print([(l,np.shape(np.array(m))) for l, m in list(groups.items())[:10]])

84
[(14, (183, 14, 24)), (22, (75, 22, 24)), (39, (6, 39, 24)), (56, (2, 56, 24)), (10, (85, 10, 24)), (20, (74, 20, 24)), (17, (129, 17, 24)), (43, (14, 43, 24)), (21, (90, 21, 24)), (34, (26, 34, 24))]


In [25]:
# train RNN
def train(model,epochs,verbose):
    for epoch in range(epochs):
        print("epoch", epoch)
        for l, matrices in groups.items():
            print("tracklists of length", l, "amount", len(matrices))
            if len(matrices) < 1 or l <= 1: continue
            batch = np.array(matrices)
            x = batch[:,:-1]
            y = batch[:,1:]
            model.fit(x,y,epochs = 1, verbose = verbose)

In [ ]:
"""train"""
train(model,50,0)

In [24]:
%%time
"""test"""
train(model,1,1)

epoch 0
tracklists of length 14 amount 183
Epoch 1/1
183/183 [==============================] - 0s - loss: 2.5351e-04     
tracklists of length 22 amount 75
Epoch 1/1
75/75 [==============================] - 0s - loss: 2.1999e-04     
tracklists of length 39 amount 6
Epoch 1/1
6/6 [==============================] - 0s - loss: 4.2460e-04
tracklists of length 56 amount 2
Epoch 1/1
2/2 [==============================] - 0s - loss: 1.0206e-04
tracklists of length 10 amount 85
Epoch 1/1
85/85 [==============================] - 0s - loss: 2.3678e-04     
tracklists of length 20 amount 74
Epoch 1/1
74/74 [==============================] - 0s - loss: 1.9470e-04     
tracklists of length 17 amount 129
Epoch 1/1
129/129 [==============================] - 0s - loss: 1.7915e-04     
tracklists of length 43 amount 14
Epoch 1/1
14/14 [==============================] - 0s - loss: 1.0739e-04
tracklists of length 21 amount 90
Epoch 1/1
90/90 [==============================] - 0s - loss: 1.4820e-04     

2/2 [==============================] - 0s - loss: 1.5144e-04
tracklists of length 77 amount 1
Epoch 1/1
1/1 [==============================] - 0s - loss: 1.9652e-04
tracklists of length 72 amount 5
Epoch 1/1
5/5 [==============================] - 0s - loss: 5.9218e-05
tracklists of length 108 amount 2
Epoch 1/1
2/2 [==============================] - 0s - loss: 4.9156e-05
tracklists of length 59 amount 2
Epoch 1/1
2/2 [==============================] - 0s - loss: 1.1180e-04
tracklists of length 267 amount 1
Epoch 1/1
1/1 [==============================] - 0s - loss: 1.9285e-04
tracklists of length 73 amount 1
Epoch 1/1
1/1 [==============================] - 0s - loss: 1.5049e-04
tracklists of length 61 amount 1
Epoch 1/1
1/1 [==============================] - 0s - loss: 9.4875e-05


In [33]:
model.save_weights("discog_rnn.h5")

In [ ]:
model.load_weights("discog_rnn.h5")

In [26]:

# generate 1 list

import random

def random_tracks(df, amount=1):
    rs = [random.randint(0,df.shape[0]) for _ in range(amount)]
    return [df.iloc[r] for r in rs]

def metric(vector1, vector2):
    v1, v2 = np.array(vector1), np.array(vector2)
    d = (v1 - v2)
    return np.sum(d * d)

def nearest_neighbor(vec, tracks):
    return min(tracks, key=lambda t: metric(vec, to_vector(t)))

def generate_next(tracks, df, model, params):
    predicted = model.predict(np.array([to_matrix(tracks)]))[0][-1]
    neighbor = nearest_neighbor(predicted, [track for i, track in df.iterrows()])
    return neighbor

def generate_next_v2(tracks, df, model, params):
    predicted = model.predict(np.array([to_matrix(tracks)]))[0][-1]
    neighbor = nearest_neighbor(predicted, random_tracks(df,params['tournament_size']))
    return neighbor



def generate_list(df, model, params):
    tracks = random_tracks(df, 1)
    yield tracks[-1]
    for i in range(1, params['amount']):
        tracks += [generate_next_v2(tracks, df, model, params)]
        yield tracks[-1]


In [32]:
params = {'amount':15, 'tournament_size': 1000}

f = open('discog_gen.txt','a+')

for i in range(5):
    gen = generate_list(tracklist_df,model, params)
    for track in gen:
        print(track['artist'],'-',track['track'])
        f.write(track['artist'] + '-' + track['track'] + "\n")
    print("--------------")
    f.write("--------------")

f.close()

Britney Spears - Do Something (Thick Vocal Mix)
Uniting Nations - Out Of Touch
Sébastien Léger - Hypnotised (DJ Spen & The MuthaFunkaz Remix)
X-Press 2 - Give It (Extended Club Mix)
Various - At The Club
Jamiroquai - Seven Days In Sunny June (Steve Mac Classic Remix)
Jamiroquai - Seven Days In Sunny June (Steve Mac Classic Remix)
Gadjo - So Many Times (Original Mix)
House Bros. - Love What You Feel (House Bros Soulful Vocal)
De Souza - Guilty (Eddie Thoneick Remix)
Identity (3) - This Girl (Miki Moto & Bobby Blanco Mix)
Jamiroquai - Seven Days In Sunny June (Steve Mac Classic Remix)
Jennifer Lopez - Get Right (Louie Vega Club Mix)
Various - CD 2 ● Workout 2
Eric Kupper - Cocoon (12" Mix)
--------------
Polar Inertia - Major Axis
Oscar Mulero - Blackstar
DBX - Losing Control
Joseph Cocherell - Poptones
Regis - Purification
The Flashbulb - Undiscovered Colours
DJ Bone - GTC (DJ Bone's Deep Concentration Mix)
Black Asteroid - Black Body Radiation
Shed - Boom Room
Lucy (12) - LX2
Grindvik 